In [1]:
# DISCLAIMER : not an investment advice.
# VaR analysis

import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf
import numpy as np
import datetime as dt
from scipy.stats import norm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import scipy.stats as stat
from IPython.core.pylabtools import figsize

In [ ]:
tickers = ['AAPL', 'GOOG', 'AMZN', 'MSFT'] # removed $FB, $NFLX
weights = np.array([.25, .25, .25, .25])
initial_investment = 1000000 # 1 mn USD
data = pdr.get_data_yahoo(tickers, start="2017-01-01", end=dt.date.today())['Close']
returns = data.pct_change()
returns.tail()

In [ ]:
cov_matrix = returns.cov()
cov_matrix

In [ ]:
avg_rets = returns.mean()
port_mean = avg_rets.dot(weights) # dot product to find mean of portfolio
port_stdev = np.sqrt(weights.T.dot(cov_matrix).dot(weights))
print (port_stdev)
mean_investment = (1 + port_mean) * initial_investment
stdev_investment = initial_investment * port_stdev

In [ ]:
cutoff = norm.ppf(0.05, mean_investment, stdev_investment) #assuming confidence level = 95%
# ppf is the inverse of CDF to map x for given f(x); here Z-value which we refer from log table book
var = initial_investment - cutoff
var

In [ ]:
var_array = []
days = [20] #since it is 0.05 VaR
for y in days:
    var_array = []
    for x in range(y + 1):
        var_array.append(np.round(var * np.sqrt(x + 1), 5))
        print(str(x) + " day VaR >> @ Confidence level 95% >> " + str(np.round(var * np.sqrt(x),2)))

    figsize(20, 7)
    plt.xlabel("Day")
    plt.ylabel("Max portfolio loss (USD)")
    plt.title("Max portfolio loss (VaR) over" + str(y) + "-day period")
    plt.show(plt.plot(var_array))

In [ ]:
for i in tickers:
    returns[i].hist(bins = 500)
    x = np.linspace(port_mean - 4 * port_stdev, port_mean + 4 * port_stdev, 1000)
    figsize(20, 7)
    plt.plot(x, norm.pdf(x, port_mean, port_stdev), "r")
    plt.title(i + " returns vs. normal distribution")
    plt.show()